<a href="https://colab.research.google.com/github/soyoungpark9187/DeepLearning/blob/master/DL_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Competition 201735831 박소영**

drive mount

In [ ]:
from google.colab import drive
  
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import os
from skimage import io
from skimage.util import random_noise
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import numpy as np
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2

In [ ]:
def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        # download pretrained weight
        state_dict = load_state_dict_from_url(model_urls[arch],progress=progress)
        # assign to model
        model.load_state_dict(state_dict)
    return model

In [ ]:
def resnet18(pretrained=False, progress=True, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', BasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)

model

In [ ]:
from torchvision import models
import torch

use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

model = models.resnet18(pretrained=True)

# last layer modification
model.fc = nn.Linear(512,10)
model.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

train

In [ ]:
def train(model, train_loader, optimizer, epoch):
  model.train()
  for batch_idx, (image, target) in enumerate(train_loader):
    data, target = image.to(DEVICE), target.to(DEVICE)
    optimizer.zero_grad()
    output = model(data)
    loss = F.cross_entropy(output, target)
    loss.backward()
    optimizer.step()

    if batch_idx % 150 == 0 :
      print('Train Epoch : {} [{}/{} ({:.0f})%]\tLoss: {:.6f}'
      .format(epoch, batch_idx*len(image),len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.item()))

def evaluate(model, test_loader):
  model.eval()
  test_loss =0
  correct =0
  with torch.no_grad():
    for (image, target) in test_loader:
      image, label = image.to(DEVICE), target.to(DEVICE)
      output = model(image)

      test_loss += F.cross_entropy(output, label, reduction='sum').item()
      pred = output.max(1, keepdim=True)[1]
      correct+= pred.eq(label.view_as(pred)).sum().item()
  
  test_loss /= len(test_loader.dataset)
  test_accuracy = 100. * correct / len(test_loader.dataset)
  return test_loss, test_accuracy

data augmentation methods

In [ ]:
def augmentation(image):
    height, width, channel = image.shape
    # random rotation degree
    degree = np.random.randint(45,90)
    # color
    b = idx = np.random.randint(255)
    g = idx = np.random.randint(255)
    r = idx = np.random.randint(255)

    # pt1 #TODO: 범위
    y1 = np.random.randint(height/2)
    x1 = np.random.randint(width/2)

    # pt2
    y2 = np.random.randint(y1, height)
    x2 = np.random.randint(x1, height)

    matrix = cv2.getRotationMatrix2D((width/2, height/2), degree, 1)

    image = cv2.warpAffine(image, matrix, (width, height))

    image = cv2.rectangle(image, (y1, x1), (y2, x1), (b, g, r), -1)

    image = random_noise(image, mode='gaussian', mean=0, var=0.1)

    return image

data load

In [ ]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

BATCH_SIZE = 16

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                        std=[0.229,0.224,0.225])
])

    
class MyDataset(Dataset):
  def __init__(self, image_dir, label, transforms=None):
    self.image_dir = image_dir
    self.label = label
    self.image_list = os.listdir(self.image_dir)
    self.transforms = transform
  
  def __len__(self):
    return len(self.image_list)
  
  def __getitem__(self,idx):
    # if torch.is_tensor(idx):
    #   idx = idx.tolist()

    image_name = os.path.join(self.image_dir, self.image_list[idx])
    image = io.imread(image_name)

    height, width, channel = image.shape
    # random rotation degree
    degree = np.random.randint(0,90)
    # color
    b = idx = np.random.randint(255)
    g = idx = np.random.randint(255)
    r = idx = np.random.randint(255)

    # pt1 #TODO: 범위
    y1 = np.random.randint(height/2)
    x1 = np.random.randint(width/2)

    # pt2
    y2 = np.random.randint(y1, height)
    x2 = np.random.randint(x1, height)

    matrix = cv2.getRotationMatrix2D((width/2, height/2), degree, 1)

    image = cv2.warpAffine(image, matrix, (width, height))

    image = cv2.rectangle(image, (y1, x1), (y2, x2), (b, g, r), -1)

    image = random_noise(image, mode='gaussian', mean=0, var=0.1)

    # type casting
    image.astype(float)
    ### transform
    image = transform(image)
    return (image,self.label)

data_dir = "/content/drive/My Drive"
root = data_dir + '/swdl2020'
#cheetah : 0 , jaguar : 1, tiger : 2, hyena : 3

# original train
cheetah_train = MyDataset(root+"/train/cheetah_train_resized",0,transform)
jaguar_train = MyDataset(root+"/train/jaguar_train_resized",1,transform)
tiger_train = MyDataset(root+"/train/tiger_train_resized",2,transform)
hyena_train = MyDataset(root+"/train/hyena_train_resized",3,transform)

train_set = ConcatDataset([cheetah_train, jaguar_train, tiger_train, hyena_train])
print("Number of Training set images : ", len(train_set))

#original val
cheetah_val = MyDataset(root+"/validation/cheetah_validation_resized",0, transform)
jaguar_val = MyDataset(root+"/validation/jaguar_validation_resized",1, transform)
tiger_val = MyDataset(root+"/validation/tiger_validation_resized",2, transform)
hyena_val = MyDataset(root+"/validation/hyena_validation_resized",3,transform)

val_set = ConcatDataset([cheetah_val, jaguar_val, tiger_val, hyena_val])
print("Numver of Validation set images : ", len(val_set))

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle = True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)

Number of Training set images :  3600
Numver of Validation set images :  400


execute

In [ ]:
import torch.optim as optim
learnning_rate = 0.01
EPOCH = 10

optimizer = optim.Adadelta(model.parameters(), lr=learnning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
loss_function = nn.CrossEntropyLoss()

for epoch in range(1, EPOCH+1):
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, val_loader)
    print('[{}] Test Loss : {:.4f}, Accuracy : {:.4f}%'.format(epoch, test_loss, test_accuracy))

Train Epoch : 1 [0/3600 (0)%]	Loss: 0.100249
Train Epoch : 1 [2400/3600 (67)%]	Loss: 0.075964
[1] Test Loss : 0.1210, Accuracy : 96.0000%
Train Epoch : 2 [0/3600 (0)%]	Loss: 0.048461
Train Epoch : 2 [2400/3600 (67)%]	Loss: 0.286370
[2] Test Loss : 0.0885, Accuracy : 98.2500%
Train Epoch : 3 [0/3600 (0)%]	Loss: 0.376815
Train Epoch : 3 [2400/3600 (67)%]	Loss: 0.094187
[3] Test Loss : 0.1022, Accuracy : 96.7500%
Train Epoch : 4 [0/3600 (0)%]	Loss: 0.301173
Train Epoch : 4 [2400/3600 (67)%]	Loss: 0.016338
[4] Test Loss : 0.0972, Accuracy : 96.7500%
Train Epoch : 5 [0/3600 (0)%]	Loss: 0.106025
Train Epoch : 5 [2400/3600 (67)%]	Loss: 0.224155
[5] Test Loss : 0.0961, Accuracy : 97.7500%
Train Epoch : 6 [0/3600 (0)%]	Loss: 0.035911
Train Epoch : 6 [2400/3600 (67)%]	Loss: 0.051092
[6] Test Loss : 0.0879, Accuracy : 97.7500%
Train Epoch : 7 [0/3600 (0)%]	Loss: 0.056041
Train Epoch : 7 [2400/3600 (67)%]	Loss: 0.098457
[7] Test Loss : 0.0995, Accuracy : 97.5000%
Train Epoch : 8 [0/3600 (0)%]	Loss

test loader

In [ ]:
class TestDataset(Dataset):
  def __init__(self, image_dir, transforms=None):
    self.image_dir = image_dir
    self.image_list = os.listdir(self.image_dir)
    self.transforms = transforms
  
  def __len__(self):
    return len(self.image_list)
  
  def __getitem__(self,idx):
    # if torch.is_tensor(idx):
    #   idx = idx.tolist()

    image_name = os.path.join(self.image_dir, self.image_list[idx])
    image = io.imread(image_name)

    ### transform
    image = transform(image)

    return (image,self.image_list[idx].split('.')[0])


test_set = TestDataset(root+"/test100", transforms)
test_loader = DataLoader(test_set)

prediction

In [ ]:
import pandas as pd

#cheetah : 0 , jaguar : 1, tiger : 2, hyena : 3
map = ['cheetah','jaguar','tiger','hyena']

model.eval()
df = pd.DataFrame(columns=['id','category'])
with torch.no_grad():
    for (image, image_name) in test_loader:
        # type casting
        image = image.type(torch.cuda.DoubleTensor)
        output = model(image)
        pred = output.max(1, keepdim=True)[1]
        df = df.append(pd.DataFrame([[image_name[0], map[pred.squeeze().tolist()]]], columns=['id','category']))
df

,id,category
0,10083,jaguar
0,10348,tiger
0,10295,tiger
0,10714,hyena
0,1062,jaguar
...,...,...
0,8986,tiger
0,9042,tiger
0,938,cheetah
0,9682,cheetah


label counting

In [ ]:
print('tiger', len(df[df['category']=='tiger']))
print('cheetah', len(df[df['category']=='cheetah']))
print('jaguar', len(df[df['category']=='jaguar']))
print('hyena', len(df[df['category']=='hyena']))

tiger 24
cheetah 23
jaguar 36
hyena 17


save csv file

In [ ]:
df.to_csv(root +'/res.csv'+'20200624', index=False)
print(df.columns)
df.head()

Index(['id', 'category'], dtype='object')


,id,category
0,10083,jaguar
0,10348,tiger
0,10295,tiger
0,10714,hyena
0,1062,jaguar


In [ ]:
# compare
answer = pd.read_excel(root+"/label.xlsx")
answer.head()

,412,jaguar
0,458,cheetah
1,509,tiger
2,938,cheetah
3,1062,jaguar
4,1076,hyena


In [ ]:
arr1  = df.values
arr2 = answer.values

In [ ]:
sum=0
for i in range(99):
  for j in range(99):
    if int(arr1[i+1][0]) == arr2[j][0]:
      if arr1[i+1][1] == arr2[j][1]:
        sum+=1

print(sum)

84
